# Projeto 1 - Ciência dos Dados

Nome: Maria Eduarda Torres

Nome: Gabriela Choichit Giosa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [455]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [456]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/duddatorres/Documents/Cdados/Projetotweeter


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [457]:
dados = pd.read_excel('Iphone_11.xlsx')
dados.head()

,Treinamento,categorias
0,rt @maionese53: papai acaba de ajeitar meu cel...,4
1,@kt_lilstar @tiktokproibido no fim qm vai ter ...,3
2,estou aceitando um iphone 11 pro max ou um s20...,3
3,ai n to fazendo nada acho q vou comprar um iph...,3
4,@_gabialmeida07 ganhando iphone 11 eu lambia a...,4


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### Separando os dados

In [458]:
GM = dados['categorias'] == 5
G = dados['categorias'] == 4
N = dados['categorias'] == 3
NG = dados['categorias'] == 2
D = dados['categorias'] == 1

In [459]:
Gosta_muito = dados.loc[GM,:]
Gosta = dados.loc[G,:]
Neutro = dados.loc[N,:]
Nao_gosta = dados.loc[NG,:]
Detesta = dados.loc[D,:]

In [460]:
universo = dados['categorias']
universo.value_counts()

3    156
5     94
4     24
2     16
1     10
Name: categorias, dtype: int64

##### Subcategorias:


In [461]:
#Tweets positivos
Ok = (dados['categorias'] == 5) | (dados['categorias'] == 4)
positivo = dados.loc[Ok,:]

In [462]:
#Tweets negativos
N_ok = (dados['categorias'] == 1) | (dados['categorias'] == 2)
negativo = dados.loc[N_ok,:]

In [463]:
#Tweets neutros
n = (dados['categorias'] == 3)
neutro = dados.loc[n,:]

##### Total de ocorrências

In [464]:
total = len(dados.index)
total_neg = len(negativo.index)
total_pos = len(positivo.index)
total_neutro = len(neutro.index)
total_pos, total, total_neg, total_neutro

(118, 300, 26, 156)

##### Probabilidade de ser positivo

In [465]:
prob_pos = total_pos / total
prob_pos

0.3933333333333333

##### Probabilidade de ser negativo

In [466]:
prob_neg = total_neg / total
prob_neg

0.08666666666666667

##### Probabilidade de ser neutro

In [467]:
prob_neutro = total_neutro / total
prob_neutro

0.52

##### Transformando em texto 

In [468]:
texto=""

In [469]:
for e in dados.Treinamento:
    texto+= " " + e 


##### Filtrando texto positivo

In [470]:
texto_positivo =""

In [471]:
for e in positivo.Treinamento:
    texto_positivo+= " " + e 


##### Filtrando texto negativo 

In [472]:
texto_negativo =""

In [473]:
for e in negativo.Treinamento:
    texto_negativo+= " " + e 
    

##### Filtrando texto neutro

In [474]:
texto_neutro =""

In [475]:
for e in neutro.Treinamento:
    texto_neutro+= " " + e 

##### Limpando textos


In [476]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

##### Split and cleanup

In [477]:
import re

texto = cleanup(texto.lower())
texto_split = pd.Series(texto.split())

texto_positivo = cleanup(texto_positivo.lower())
texto_positivo_split = pd.Series(texto_positivo.split())

texto_negativo = cleanup(texto_negativo.lower())
texto_negativo_split = pd.Series(texto_negativo.split())

texto_neutro= cleanup(texto_neutro.lower())
texto_neutro_split = pd.Series(texto_neutro.split())

for e in dados.Treinamento:
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
x

'rt   eu quero pega o iphone 11 cansei desse já'

##### Frequencia absoluta

In [478]:
tabela_texto = texto_split.value_counts()

tabela_texto_positivo = texto_positivo_split.value_counts()

tabela_texto_negativo = texto_negativo_split .value_counts()

tabela_texto_neutro = texto_neutro_split.value_counts()

soma_palavras=tabela_texto.sum()
soma_positivo=tabela_texto_positivo.sum()
soma_negativo=tabela_texto_negativo.sum()
soma_neutro=tabela_texto_neutro.sum()

print(soma_palavras,soma_positivo,soma_negativo,soma_neutro)


probabilidade_pos=soma_positivo/ soma_palavras
probabilidade_neg=soma_negativo/ soma_palavras
probabilidade_neu=soma_neutro/ soma_palavras

print(probabilidade_pos,probabilidade_neg,probabilidade_neu)

5430 1703 551 3176
0.3136279926335175 0.10147329650092081 0.5848987108655617


##### Frequencias relativas

In [479]:
tabela_texto_relativa = texto_split.value_counts(True)

In [480]:
tabela_positivo_relativa = texto_positivo_split.value_counts(True)
tabela_positivo_relativa

iphone          0.073987
11              0.070464
                  ...   
@delcacomani    0.000587
suggar          0.000587
Length: 575, dtype: float64

In [481]:
tabela_negativo_relativa = texto_negativo_split.value_counts(True)

In [482]:
tabela_neutro_relativa = texto_neutro_split.value_counts(True)

##### Probalidade

In [483]:
a=tabela_positivo_relativa.get('caro', 0)
b=tabela_negativo_relativa.get('caro', 0)
c=tabela_neutro_relativa.get('caro', 0)
a,b,c

(0.0005871990604815032, 0.0054446460980036296, 0)

##### Interseção entre os conjuntos

In [484]:
set_positivo = set(tabela_positivo_relativa.index)
set_negativo = set(tabela_negativo_relativa.index)
set_neutro = set(tabela_neutro_relativa.index)

inter = set_positivo.intersection(set_negativo, set_neutro)

##### Frase a se classificar

In [485]:
base_teste = pd.ExcelFile('Iphone_11.xlsx')
df1 = pd.read_excel(base_teste,'Teste')
df1.iloc[99]


Teste         rt @_matheusmaki: queria um iphone 11 e um app...
Unnamed: 1                                                    4
Name: 99, dtype: object

##### Limpando tweets e classificando

In [486]:


i=0
for e in df1.Teste: 
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
    tabela_tweets = pd.Series(x.split())
    
    positive=1
    for palavra in tabela_tweets:
        positive*=tabela_positivo_relativa.get(palavra, 1)

    negative=1    
    for palavra in tabela_tweets:
        negative*=tabela_negativo_relativa.get(palavra, 1)
    
    cinza=1
    for palavra in tabela_tweets:
        cinza*=tabela_neutro_relativa.get(palavra, 1)
  
    
    positive=probabilidade_pos*positive
    negative=probabilidade_neg*negative
    cinza=probabilidade_neu*cinza
    
    if positive>negative and positive>cinza:
        classificador=5
    elif negative>positive and negative>cinza:
        classificador=1
    elif cinza>positive and cinza>negative:
        classificador=3
        
    #df1.insert(1, "classificador", classificador, True)
    df1.loc[i, "classificador"]=classificador
    i+=1
    
print(positive, negative,cinza)

print (df1)
    

1.1789495075854912e-10 1.4277847629877094e-11 5.3396811889157235e-11
                                                 Teste  Unnamed: 1  \
0    rt @esteziinh_: @biaaa_crf marco mesmo pq não ...           4   
1    gabrieljnio6 @ericat_lol po man o meu é iphone...           3   
..                                                 ...         ...   
102  rt @euleozin_: troco por iphone 11 pro, chamar...           3   
103            o auto falante do iphone 11 pro é otimo           5   

     classificador  
0              1.0  
1              3.0  
..             ...  
102            5.0  
103            5.0  

[104 rows x 3 columns]


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**